In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

# torch.nn.functional (F) 提供了许多神经网络相关的函数，主要用途包括：
# 1. 激活函数：如 F.relu(), F.sigmoid(), F.tanh(), F.softmax() 等
# 2. 损失函数：如 F.cross_entropy(), F.mse_loss(), F.binary_cross_entropy() 等  
# 3. 池化操作：如 F.max_pool2d(), F.avg_pool2d() 等
# 4. 卷积操作：如 F.conv2d(), F.conv1d() 等
# 5. 正则化：如 F.dropout(), F.batch_norm() 等
# 6. 其他工具函数：如 F.interpolate(), F.pad() 等

# 与 nn.Module 的区别：
# - nn.Module 类（如 nn.ReLU()）会创建一个层对象，可以保存参数和状态
# - F 中的函数是无状态的，直接对输入进行计算
# 例如：nn.ReLU() 创建层对象，F.relu() 直接计算ReLU激活

In [2]:
net=nn.Sequential(
    nn.Linear(20,256),
    nn.ReLU(),
    nn.Linear(256,10)
)
X=torch.rand(2,20)
net(X)

tensor([[ 6.3380e-02, -4.5030e-02, -3.4262e-01,  1.3780e-01,  4.8624e-02,
          2.3667e-02, -3.2229e-02, -3.6548e-03,  5.3033e-02, -2.1022e-05],
        [ 8.1893e-02, -8.3103e-02, -2.9704e-01,  2.0120e-01,  1.2419e-01,
         -1.0935e-02, -5.4518e-02,  3.5035e-02,  2.4709e-02, -7.4179e-02]],
       grad_fn=<AddmmBackward0>)

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    def forward(self,X):
        X=F.relu(self.hidden(X))
        return self.out(X)

In [6]:
net=MLP()
net(X)

tensor([[ 0.0681,  0.1961, -0.0464,  0.0142,  0.1817, -0.1537, -0.2041,  0.2387,
          0.1422,  0.1306],
        [ 0.0082,  0.2044, -0.1228,  0.0325,  0.2877, -0.1420, -0.1651,  0.0793,
          0.0635,  0.1157]], grad_fn=<AddmmBackward0>)

In [7]:
# enumerate() 是Python内置函数，用于枚举（遍历）可迭代对象
# 它返回一个枚举对象，包含索引和对应的值
# 语法：enumerate(iterable, start=0)
# 例如：enumerate(['a', 'b', 'c']) 返回 [(0, 'a'), (1, 'b'), (2, 'c')]

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # enumerate(args) 会返回 (索引, 模块) 的元组
        # 例如：args = (nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
        # enumerate(args) 返回：(0, nn.Linear(20,256)), (1, nn.ReLU()), (2, nn.Linear(256,10))
        for idx, module in enumerate(args):
            # 将每个模块以字符串索引的形式存储在 _modules 字典中
            # _modules 是 nn.Module 的特殊属性，用于存储子模块
            self._modules[str(idx)] = module
    
    def forward(self, X):
        # 按顺序执行每个模块
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0503, -0.1602, -0.3288, -0.0421, -0.0363,  0.0764, -0.1828,  0.1323,
         -0.1076, -0.0684],
        [-0.0863, -0.0531, -0.3293, -0.1453,  0.0282, -0.0884, -0.0298, -0.0457,
         -0.2598,  0.0314]], grad_fn=<AddmmBackward0>)

In [ ]:
# FixedHiddenMLP 类演示了一个具有固定权重和控制流的神经网络
# 这个类展示了以下几个重要概念：

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 创建一个固定的随机权重矩阵，不参与梯度计算
        # requires_grad=False 表示这个参数不会在反向传播中更新
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        # 创建一个可训练的线性层
        self.linear=nn.Linear(20,20)
    
    def forward(self,X):
        # 第一次线性变换（可训练参数）
        X=self.linear(X)
        # 使用固定权重进行矩阵乘法，然后加1并应用ReLU激活
        # 这里展示了如何在网络中使用固定的、不可训练的参数
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        # 第二次线性变换（复用同一个线性层）
        X=self.linear(X)
        # 控制流：当张量绝对值之和大于1时，持续除以2
        # 这展示了神经网络中可以包含条件语句和循环
        while X.abs().sum()>1:
            X/=2
        # 返回所有元素的和（标量输出）
        return X.sum()

# 创建网络实例并测试
net=FixedHiddenMLP()
net(torch.rand(2,20))

tensor(0.0504, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                               nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))
chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(torch.rand(2,20))

tensor(0.1571, grad_fn=<SumBackward0>)